<a href="https://colab.research.google.com/github/raghavmundra/Covid_Data_Visualiztion/blob/main/covid_data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Importing required libraries

In [1]:
import pandas as pd
from bs4 import *
import requests
import json

##Webscrapping data

In [2]:
#Fetching covid vaccination data
path1='https://www.mygov.in/sites/default/files/covid/vaccine/vaccine_counts_today.json'
page1 = requests.get(path1)
soup1 = BeautifulSoup(page1.content, 'html.parser')
site_json=json.loads(soup1.text)
site_json['vacc_st_data'][0]
vaccine_data=pd.DataFrame(columns=['state','dose1','dose2','total_doses'])
c=0
for i in site_json['vacc_st_data']:
  vaccine_data.loc[c]=[i['covid_state_name'],i['dose1'],i['dose2'],i['total_doses']]
  c+=1
#Fetching covid cases data
path2='https://www.mygov.in/sites/default/files/covid/covid_state_counts_ver1.json'
page2 = requests.get(path2)
soup2 = BeautifulSoup(page2.content, 'html.parser')
site_json2=json.loads(soup2.text)
active_cases=site_json2['Active']
state_names=site_json2['Name of State / UT']
total_deaths=site_json2['Death']
cured_cases=site_json2['Cured/Discharged/Migrated']
total_cases=site_json2['Total Confirmed cases']
covid_data=pd.DataFrame(columns =['state', 'total_cases','active_cases','discharged_cases','total_deaths'])
covid_data.loc[0]=[state_names['0'],total_cases['0'],active_cases['0'],cured_cases['0'],total_deaths['0']]
for i in range(0,36):
  covid_data.loc[i]=[state_names[str(i)],total_cases[str(i)],active_cases[str(i)],cured_cases[str(i)],total_deaths[str(i)]]

##Data Exploration

In [3]:
covid_data

,state,total_cases,active_cases,discharged_cases,total_deaths
0,Andaman and Nicobar,7646,6,7511,129
1,Andhra Pradesh,2062781,5398,2043050,14333
2,Arunachal Pradesh,55075,141,54654,280
3,Assam,608126,3866,598296,5964
4,Bihar,726045,30,716354,9661
5,Chandigarh,65320,28,64472,820
6,Chhattisgarh,1005799,214,992013,13572
7,Dadra and Nagar Haveli and Daman and Diu,10679,4,10671,4
8,Delhi,1439526,340,1414095,25091
9,Goa,177819,600,173862,3357


In [4]:
vaccine_data

,state,dose1,dose2,total_doses
0,Andaman and Nicobar,292988,186655,479643
1,Andhra Pradesh,31444989,18032266,49477255
2,Arunachal Pradesh,768011,507279,1275290
3,Assam,19853518,7072829,26926347
4,Bihar,48394063,15912408,64306471
5,Chandigarh,921706,521520,1443226
6,Chhattisgarh,14415425,6646148,21061573
7,Dadra and Nagar Haveli and Daman and Diu,657198,352146,1009344
8,Delhi,12867945,7104856,19972801
9,Goa,1252224,851685,2103909
